In [4]:
import credentials # Api key is stored in this file, remove to avoid errors if you clone from github

import pvdeg
import pvlib
import os 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
from scipy.linalg import cholesky
from scipy import stats

First get weather data and metadata for a desired location (latitude and logitude)

In [2]:
# change to desired values (currently Miami)
latitude = 25.783388
longitude = -80.189029

API_KEY = credentials.API_KEY # my personal NREL api key
email ='tobin.ford@nrel.gov' # replace these values with your appropriate information and remove or comment out first line of first block (import credentials)

# reads NSRDB data 
weather_df, meta = pvlib.iotools.get_psm3(latitude, longitude, API_KEY, email, names='2019', map_variables=True)

c:\ProgramData\anaconda3\envs\monte-carlo\lib\site-packages\pvlib\iotools\psm3.py:183: pvlibDeprecationWarning: The ``get_psm3`` function will default to leap_day=True starting in pvlib 0.11.0. Specify leap_day=True to enable this behavior now, or specify leap_day=False to hide this warning.
  warnings.warn(


User has 3 parameters for initial implementation: See Kempe's "Deg Miami" tab in excel<br>



activation energy, Ea <br>
irradiance relation, x<br>
ln(R0)<br>

|           |   Ea   |   x   | ln(R0) |
|:---------:|:-----:|:----:|:-------:|
|   **Ea**  |   1   |   a  |   b     |
|   **x**   |   a   |   1  |   c     |
| **ln(R0)**|   b   |   c  |   1     |

Notice symmetry across diagonal <br>

In [3]:
# USER ENTERED VALUES
# Correlation Coefficients
Ea_X = 0.0269
Ea_lnR0 = -0.9995 
X_lnR0 = -0.0400

# Activation Energy
mean_Ea = 62.08 # average
sd_Ea = 7.3858 # standard deviation

# Irradiance relation
mean_X = 0.0341 # average
sd_X = 0.0992757 # standard deviation

# ln(R0)
mean_lnR0 = 13.7223084 
sd_lnR0 = 2.47334772

# number of iterations
n = 20000


In [54]:
# notice symmetry of matrix
A = np.array([[1,   Ea_X,   Ea_lnR0],
              [Ea_X,    1,   X_lnR0],
              [Ea_lnR0, X_lnR0,   1]])

# conceptually similar to the square root of a matrix
A_decomp = cholesky(A, lower=True) # does it need to be upper, either way lnR0 values are very wrong

# generating standard distribution of points for each
ea, x, lnR0 = np.random.normal(mean_Ea, sd_Ea, 20_000), np.random.normal(mean_X, sd_X, 20_000), np.random.normal(mean_lnR0, sd_lnR0, 20_000)


# an attempt to try what I am seeing in the excel macro data
# only ea has standard distribution based on mean and sd
# x and lnR0 have the regular standard distribution to be used in decomposition?
# this mimics the behavior in the macro
'''
ea = np.random.normal(mean_Ea, sd_Ea, n) 
x = np.random.normal(loc=0, scale=1, size=n)
lnR0 = np.random.normal(loc=0, scale=1, size=n)
'''

# transpose built in so I don't have to do it with another function
samples_matrix = np.array([ea, x, lnR0])

data = {
    'ea': ea,
    'x': x,
    'lnR0': lnR0
}

uncorrelated_df = pd.DataFrame(data)


In [55]:
uncorrelated_df.head(10)

,ea,x,lnR0
0,65.436310,0.109899,16.073555
1,67.577418,0.224461,12.609616
2,59.520243,0.049271,8.701448
3,64.884445,0.033400,11.466110
4,60.103077,-0.030550,8.751237
5,57.697049,-0.007269,15.062372
6,66.804853,-0.011576,14.800246
7,46.705815,-0.037351,8.561638
8,68.157040,0.199079,14.656449
9,82.685820,-0.000534,18.555199


In [56]:
# these are one trial of kempe's before he does the decomp
# Ea: 63.51
# X: 0.7078
# LnR0; -0.324

# now my data matches his pre-decomp, changed distributions (see above block + comments)

In [57]:
print(ea.min())
print(ea.max())
print()

print(x.min())
print(x.max())
print()

print(lnR0.min())
print(lnR0.max())
print()

30.108150304452433
90.23234930834343

-0.4140115868850577
0.4238940311497664

4.903878104886665
23.84349039860878



In [58]:
# correlated stats pre-input to function

# not entirely sure what I should do here,
# do i multiply the transpose by the lower cholesky to get correlated input data for the equation on the spreadsheet
# using MonteCarloEaLnRoX

# this gives us correlated input samples
# looks like what kempe's code was doing in "MonteCarloEaLnRoX" but it is hard to tell what's happening in his code
correlated_samples = np.matmul(A_decomp, samples_matrix)

correlated_df = pd.DataFrame(correlated_samples.T, columns=['ea', 'x', 'lnR0'])

sol_pos = pvdeg.spectral.solar_position(weather_df, meta)
poa_irradiance = pvdeg.spectral.poa_irradiance(weather_df, meta)
temp_mod = pvdeg.temperature.module(weather_df=weather_df, meta=meta, poa=poa_irradiance, conf='open_rack_glass_polymer')
# we only care about global irradiance in this case


In [59]:
print(correlated_df)

              ea         x       lnR0
0      65.436310  1.870096 -64.942611
1      67.577418  2.042212 -67.183806
2      59.520243  1.650348 -59.240796
3      64.884445  1.778779 -64.522571
4      60.103077  1.586233 -59.820859
...          ...       ...        ...
19995  55.028775  1.492730 -54.660144
19996  56.077415  1.583586 -55.625351
19997  57.378449  1.505696 -56.941532
19998  58.900971  1.854478 -58.463369
19999  50.474538  1.259043 -49.955119

[20000 rows x 3 columns]


In [60]:
# attempting to compare my function input values to kempe's from macro
# Ea min:30.187
# Ea max:89.94

# lnR0 min: 4.477
# lnR0 max: 24.31

# X min:-0.387
# X max: 0.415

# uncorrelated samples of mine, random but should be sim
'''
ea
34.50720101130711
95.14753086962446

lnR0
4.258833206368452
23.659585717113735

x
-0.349302212696885
0.4300323062342357
'''
# these are very similar to kempes that he was feeding to the arrhenius equation in the macro, not similar to my correlated variables at all?
# what happens if i feed my uncorrelated variables to the function


print(correlated_df['ea'].min())
print(correlated_df['ea'].max())

print(correlated_df['lnR0'].min())
print(correlated_df['lnR0'].max())

print(correlated_df['x'].min())
print(correlated_df['x'].max())

# can conclude there is something weird happening with my input data. 

30.108150304452433
90.23234930834343
-89.72696392268183
-29.66826442680476
0.7630929838647409
2.612313547689967


In [61]:
# i know parameters I wrong but leaving for testing sake
def mikeArrhenius(weather_df, samples):

    #['Ea', 'X', 'lnR0']
    #d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + weather_df['temp_air'])) * (weather_df['ghi'] / 1000 ** y)).mean() for x, y, z in zip(samples['ea'], samples['x'], np.log(samples['lnR0']))]
    
    d = [(np.exp(z) * np.exp((x / 8.31446261815324E-03) / (273.15 + temp_mod)) * (poa_irradiance['poa_global'] / (1000 ** y))).mean() for x, y, z in zip(samples['ea'], samples['x'], np.log(samples['lnR0']))]
    # tried using cell temp
    # tried using poa

    return pd.DataFrame(d)

In [62]:
temp2 = mikeArrhenius(weather_df=weather_df, samples=uncorrelated_df) 

# trying with uncorrelated variables

In [53]:
temp2.head(24) 
# each row shows the degredation rate for that given monte carlo trial iteration
# data frame indexes are integers corresponding to each trial

,0
0,3.356155e+12
1,1.161637e+18
2,NaN
3,NaN
4,NaN
5,1.293756e+13
6,NaN
7,NaN
8,NaN
9,NaN
